In [4]:
import pandas as pd
from scipy.signal import find_peaks
import plotly.graph_objects as go
import numpy as np
import itertools
# from peakdetect import peakdetect
import os
import matplotlib.pyplot as plt

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !ls "/content/drive/MyDrive/PHYS 243W/RLC Chaos Lab/data at 279.800kHz/*/*.05.csv"

In [2]:
storage_dict = {} # initalise dictonary to save keys = volts_strings and the corresponding_list

# Import data

In [5]:
folders = [0.24,   11.00 , 13.00 , 15.00  ,17.25 , 19.25 , 3.25	,5.50 , 7.75,
0.25 ,  11.25  ,13.25,  15.25,  1.75  , 19.50 , 3.50,	5.75 , 8.00,
0.50 ,  11.50 , 13.50 , 15.50  ,17.50  ,19.75  ,3.75	,6.00 ,8.25,
0.75  , 11.75  ,13.75  ,15.75 , 17.75,  19.99 , 4.00,	6.25,  8.50,
1.00  , 12.00,  14.00 , 16.00 , 18.00 , 2.00  , 4.25,	6.50 , 8.75,
10.00,  12.25 , 14.25,  16.25,  18.25,  2.25,   4.50,	6.75 , 9.00,
10.25,  1.25  , 14.50,  16.50 , 18.50 , 2.50,   4.75,	7.00 , 9.25,
10.50 , 12.50 , 14.75,  16.75,  18.75 , 2.75,   5.00,	7.25  ,9.50,
10.75 , 12.75 , 1.50  , 17.00  ,19.00 , 3.00,   5.25,	7.50,  9.75]

In [6]:
folders = np.sort(folders)
folders

array([ 0.24,  0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,
        2.25,  2.5 ,  2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ,  4.25,
        4.5 ,  4.75,  5.  ,  5.25,  5.5 ,  5.75,  6.  ,  6.25,  6.5 ,
        6.75,  7.  ,  7.25,  7.5 ,  7.75,  8.  ,  8.25,  8.5 ,  8.75,
        9.  ,  9.25,  9.5 ,  9.75, 10.  , 10.25, 10.5 , 10.75, 11.  ,
       11.25, 11.5 , 11.75, 12.  , 12.25, 12.5 , 12.75, 13.  , 13.25,
       13.5 , 13.75, 14.  , 14.25, 14.5 , 14.75, 15.  , 15.25, 15.5 ,
       15.75, 16.  , 16.25, 16.5 , 16.75, 17.  , 17.25, 17.5 , 17.75,
       18.  , 18.25, 18.5 , 18.75, 19.  , 19.25, 19.5 , 19.75, 19.99])

In [27]:
# path = "/Users/vashisth/Desktop/PHY_243W/data_at_279.800kHz/"
path = "."
# file_path_ls = ['/content/drive/MyDrive/PHYS 243W/RLC Chaos Lab/data at 279.800kHz/{0:.2f}/{1:.2f}_05.csv'.format(x,x)  for x in folders]
file_path_ls = [path+'{0:.2f}/{1:.2f}_05.csv'.format(x,x)  for x in folders]

file_path_ls[0]

'.0.24/0.24_05.csv'

In [28]:
def define_groups(l, threshold = 5):
    groups = {}
    its = 0
    for i in l:
        numbers_except = list(l)
        del numbers_except[(l.index(i))]
        for x in range(len(numbers_except)):
            if difference(i, numbers_except[x]) < threshold:
                try:
                    groups[its].append(numbers_except[x])
                except:
                    groups[its] = [numbers_except[x]]
                del l[l.index(numbers_except[x])]
        try:
            groups[its].append(i)
        except:
            groups[its] = [i]
        its += 1
    return(groups)

def difference(a, b):
    return abs(a-b)

In [29]:
def run_one(file_path, volt_value):
  rlc_data = pd.read_csv(filepath_or_buffer =file_path) # first file

  if rlc_data.columns[0] =='Frequency': # had error from 16.50 onwards
    rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})

  time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
  time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float)
  time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float)

  #indices = find_peaks(time_series_B,  prominence=1 )[0]
  indices = find_peaks(time_series_B,  prominence=0, distance=110)[0]

  peaks_channel_A =  [time_series_A[j] for j in indices]
  peaks_channel_B =  [time_series_B[j] for j in indices]

  tuples_A_B = list( zip(peaks_channel_A, peaks_channel_B) )
  tuples_A_B = sorted(tuples_A_B, key=lambda x: x[0])

  dict_groups = define_groups(sorted( list( peaks_channel_A) ) )
  keys = list ( dict_groups.keys() )

  peaks_channel_A_unique = [ i for i in peaks_channel_A ]
  for i in range( len(peaks_channel_A) ) :
      for j in keys:
          if peaks_channel_A_unique[i] in dict_groups[j]:
              peaks_channel_A_unique[i] = np.mean(dict_groups[j])
          else:
              continue

  tuples_A_B_rounded = list( zip(peaks_channel_A_unique, peaks_channel_B) )
  tuples_A_B_rounded = sorted(tuples_A_B_rounded, key=lambda x: x[0])
  
  storage_dict = {}
  volts_string = str(volt_value)
  corresponding_list = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded))]
  
  storage_dict[volts_string] = corresponding_list
  return storage_dict

In [30]:
def run_all(ls_path, ls_volts):
  store_all = {}
  for i in range(len(ls_path)):
    path = ls_path[i]
    volt_value = ls_volts[i]
    store_one = run_one(file_path = path, volt_value = volt_value)
    store_all.update(store_one)
  
  return store_all

In [31]:
run_one(file_path_ls[66],folders[66])

FileNotFoundError: [Errno 2] No such file or directory: '.16.50/16.50_05.csv'

In [32]:
dic = run_all(file_path_ls,folders)

FileNotFoundError: [Errno 2] No such file or directory: '.0.24/0.24_05.csv'

In [3]:
keys, vals , vals_len= [], [], []
for k in dic.keys():
  keys.append(k)
  vals.append(dic.get(k))
  vals_len.append(len(dic.get(k)))
  # print(k + "  " + str(len(dic.get(k))) )


NameError: ignored

In [ ]:
plt.plot(keys, vals_len)

In [4]:
vals[i][0] 

plt_lsx = []

for j in range(len(vals)):
  ls = [vals[j][i][0] for i in range(len(vals[j])) ]
  for k in range(len(ls)):
    plt_ls.append([float(keys[j]), ls[k]])


x_axis = [i[0] for i in plt_ls]  
y_axis = np.asarray([i[1] for i in plt_ls] ) 

NameError: ignored

In [ ]:
# x_axis
y_axis [0:4] = y_axis [0:4] *( 1/100)
y_axis

In [5]:
y_axis

NameError: ignored

In [ ]:
plt.scatter(x_axis[5:],y_axis[5:])

In [ ]:
 # Check
vals[60]

In [6]:
rlc_data_1 = pd.read_csv(filepath_or_buffer ='/content/drive/MyDrive/PHYS 243W/RLC Chaos Lab/data at 279.800kHz/16.50/16.50_05.csv') # first file

#rlc_data_2 = pd.read_csv(filepath_or_buffer ='/content/drive/MyDrive/PHYS 243W/RLC Chaos Lab/data at 279.800kHz/0.24/0.24_06.csv') # second file
#rlc_data_2 = rlc_data_2.drop(axis=0, index=0)

NameError: ignored

In [7]:

#rlc_data = rlc_data_1.append(rlc_data_2, ignore_index=True)
rlc_data = rlc_data_1

NameError: ignored

In [ ]:
if rlc_data.columns[0] =='Frequency':
  rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})

In [8]:
rlc_data

NameError: ignored

In [ ]:
time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float)
time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float)

## Draw from sanity check 

In [9]:
fig = go.Figure(data=go.Scatter(
    x = time_values,
    y = time_series_B,
    mode = 'markers'
))

fig.show()

NameError: ignored

# Find and draw peaks

In [10]:
indices = find_peaks(time_series_B,  prominence=0, distance=110)[0]
indices

NameError: ignored

In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series_B,
    mode='lines+markers',
    name='Channel A'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_B[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks (A)'
))

fig.show()

NameError: ignored

## Drawing peaks of A and corresponding values of channel B

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series_A,
    mode='lines+markers',
    name='Channel A'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_A[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks (A)'
))

fig.add_trace(go.Scatter(
    y=time_series_B,
    mode='lines+markers',
    name='Channel B'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_B[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='circle'
    ),
    name='B values corr to Peak A'
))

fig.show()

NameError: ignored

# Getting the tuple of the values of peaks

In [ ]:
peaks_channel_A =  [time_series_A[j] for j in indices]
peaks_channel_B =  [time_series_B[j] for j in indices]

In [ ]:
tuples_A_B = list( zip(peaks_channel_A, peaks_channel_B) )
tuples_A_B = sorted(tuples_A_B, key=lambda x: x[0])
tuples_A_B

[(-2.409131, -0.4913785),
 (-2.291635, -0.4906766),
 (-2.27485, -0.4899442),
 (-2.229377, -0.4876247),
 (-2.22602, -0.491287),
 (-2.224494, -0.4934843),
 (-2.220222, -0.492111),
 (-2.203436, -0.4870144),
 (-2.190619, -0.4931486),
 (-2.186956, -0.4897305),
 (-2.161016, -0.4917142),
 (-2.15949, -0.4866787),
 (-2.159184, -0.4931486),
 (-2.154607, -0.4930265),
 (-2.140873, -0.4891507),
 (-2.136601, -0.490524),
 (-2.111575, -0.4922941),
 (-2.10242, -0.4847865),
 (-2.097842, -0.4921415),
 (-2.058168, -0.4918363),
 (-2.056337, -0.4880825),
 (-2.038636, -0.4898831),
 (-2.035279, -0.4867397),
 (-2.025513, -0.4866787),
 (1.468551, 0.5001984),
 (1.522568, 0.4914396),
 (1.587573, 0.4947356),
 (1.607105, 0.5008087),
 (1.609241, 0.4951018),
 (1.613513, 0.4983367),
 (1.613513, 0.4957427),
 (1.619007, 0.5024872),
 (1.626636, 0.5020905),
 (1.627247, 0.5018464),
 (1.64861, 0.5019074),
 (1.650441, 0.4991302),
 (1.661428, 0.5014191),
 (1.682485, 0.5020295),
 (1.684927, 0.4990692),
 (1.69042, 0.4980316),
 

# Get unique ones 
Notice that 62.5 and 63 are basically the same peak but we cannot just take the unique set because these two would come as different values. So we first write a function where the values are clubbed together if they are within a certain threshold

In [ ]:
def define_groups(l, threshold = 5):
    groups = {}
    its = 0
    for i in l:
        numbers_except = list(l)
        del numbers_except[(l.index(i))]
        for x in range(len(numbers_except)):
            if difference(i, numbers_except[x]) < threshold:
                try:
                    groups[its].append(numbers_except[x])
                except:
                    groups[its] = [numbers_except[x]]
                del l[l.index(numbers_except[x])]
        try:
            groups[its].append(i)
        except:
            groups[its] = [i]
        its += 1
    return(groups)

def difference(a, b):
    return abs(a-b)

In [ ]:
dict_groups = define_groups(sorted( list( peaks_channel_A) ) )
dict_groups

{0: [2.679525, 3.209021, 8.348338, 9.105197, 9.105197, 9.540086, 2.57271],
 1: [33.82824, 34.35926, 36.2804, 29.72198],
 2: [43.82153, 45.57329, 43.6964]}

### if they are in the same group replace by the mean

In [ ]:
peaks_channel_A[1]

33.82824

In [ ]:
keys = list ( dict_groups.keys() )

In [ ]:
peaks_channel_A_unique = [ i for i in peaks_channel_A ]
for i in range( len(peaks_channel_A) ) :
    for j in keys:
        # print(j)
        # print( peaks_channel_A[i] )
        # print(j)
        if peaks_channel_A_unique[i] in dict_groups[j]:
            peaks_channel_A_unique[i] = np.mean(dict_groups[j])
        else:
            continue

In [ ]:
print( "original: ", peaks_channel_A)
print( "unique: ", peaks_channel_A_unique)

original:  [8.348338, 33.82824, 2.679525, 45.57329, 9.540086, 34.35926, 2.57271, 43.82153, 9.105197, 36.2804, 3.209021, 43.6964, 9.105197, 29.72198]
unique:  [6.365724857142857, 33.54747, 6.365724857142857, 44.36374000000001, 6.365724857142857, 33.54747, 6.365724857142857, 44.36374000000001, 6.365724857142857, 33.54747, 6.365724857142857, 44.36374000000001, 6.365724857142857, 33.54747]


In [ ]:
tuples_A_B_rounded = list( zip(peaks_channel_A_unique, peaks_channel_B) )
tuples_A_B_rounded = sorted(tuples_A_B_rounded, key=lambda x: x[0])
tuples_A_B_rounded

[(6.365724857142857, 4.892727),
 (6.365724857142857, 4.893033),
 (6.365724857142857, 4.892727),
 (6.365724857142857, 4.891812),
 (6.365724857142857, 4.891812),
 (6.365724857142857, 4.890896),
 (6.365724857142857, 4.88876),
 (33.54747, 4.876247),
 (33.54747, 4.875332),
 (33.54747, 4.873501),
 (33.54747, 4.87228),
 (44.36374000000001, 4.877773),
 (44.36374000000001, 4.878078),
 (44.36374000000001, 4.875027)]

### finally group the tuples with the same first coordinate together

In [ ]:
f = lambda x:x[0]
x = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded, key=f), key=f)]
x

[(6.365724857142857, 4.892727),
 (33.54747, 4.876247),
 (44.36374000000001, 4.877773)]

In [ ]:
volts_string = '7 Volts'
corresponding_list = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded, key=f), key=f)]

In [ ]:
print("Number of points for the bifurication diagram are : {}".format(len(x)))

Number of points for the bifurication diagram are : 3


## Getting dictionary

In [ ]:
storage_dict[volts_string] = corresponding_list

In [ ]:
storage_dict

{'7 Volts': [(6.365724857142857, 4.892727),
  (33.54747, 4.876247),
  (44.36374000000001, 4.877773)]}